# Notebook for evaluating models after training on HPC

In [7]:
# load keras model and evaluate on valid set

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import json
import os
from helpers import *
print('Tensorflow version: ' + tf.__version__)
print("GPU", "available, YES" if tf.config.list_physical_devices("GPU") else "no GPU")



Tensorflow version: 2.13.0
GPU available, YES


In [8]:
# load valid data
path = '/Users/marius/Documents/Uni/TU_Berlin_Master/Masterarbeit/Dataset/valid_tfrecords/'
paths = glob.glob(path + '*.tfrecords')
valid_dataset = tf.data.TFRecordDataset(paths[6])
valid_dataset = valid_dataset.map(decode_tf_records)
valid_dataset = valid_dataset.batch(8)



In [9]:
# paths to log folders 
paths = glob.glob('/Users/marius/Documents/Uni/TU_Berlin_Master/Masterarbeit/HPC_files/logs/*')

for path in paths:
    print(path)
    # join path and string
    model_path = path + '/model.keras'
    config_path = path + '/config.json'
    
    # load model
    model =  keras.models.load_model(model_path, compile=False)
    # load json config
    with open(config_path, 'r') as fp:
      config = json.load(fp)

    # make dir in logs
    save_dir = path + '/__eval/'
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # get one speechfile from dataset
    speech_for_predicition = set_1_speechfile(valid_dataset, save_dir, config)
    # predict
    y_pred_tf = model.predict(speech_for_predicition)
    # change shape to (len(audio), 1)
    y_pred_np = tf.squeeze(y_pred_tf, axis=-1).numpy()
    
    # save plot to disk
    plt.figure(figsize=(8, 4))
    x = np.arange(0, len(y_pred_np)/int(config['sr']), 1/int(config['sr']))
    plt.plot(x, y_pred_np)
    plt.title('predicted')
    plt.xlabel('Time in s')
    plt.ylabel('Amplitude')
    plt.savefig(save_dir + '/_predicted.png')
    plt.close()
    
    # save audiofile with tensorflow
    y_pred_tf = tf.squeeze(y_pred_tf, axis=-1)
    audio_tf = tf.audio.encode_wav(y_pred_tf, int(config['sr']))
    tf.io.write_file(save_dir + '/_predicted' + '.wav', audio_tf)



/Users/marius/Documents/Uni/TU_Berlin_Master/Masterarbeit/HPC_files/logs/20231021-083045_v81


2023-10-21 12:25:10.073962: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4135/4135 [==============================] - 48s 11ms/step
/Users/marius/Documents/Uni/TU_Berlin_Master/Masterarbeit/HPC_files/logs/20231021-085301_v82


2023-10-21 12:25:59.758182: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4135/4135 [==============================] - 46s 11ms/step
